In [34]:
device = "cuda:5"

In [35]:
from ultralytics import YOLO
detection_model = YOLO("yolo12x.pt").to(device).eval()

In [51]:
model_names = [
    None, None
] + [
    ("bmshj2018-factorized", i) for i in range(1, 9)
]

In [52]:
from PIL import Image
from torchvision import transforms
import torch as th

def compress_image(input_path, output_path, model=None):
    img = Image.open(input_path).convert("RGB")
    x = transforms.ToTensor()(img).unsqueeze(0).to(device)

    if model is not None:
        with th.no_grad():
            x = model(x)["x_hat"].clamp(0, 1)

    reconstructed_img = transforms.ToPILImage()(x.squeeze(0))
    reconstructed_img.save(output_path)

In [53]:
from compressai.zoo import models
import os
from tqdm.notebook import tqdm

for model_name in model_names:
    compress_model = None
    if model_name is not None:
        compress_model = models[model_name[0]](quality=model_name[1], pretrained=True).to(device).eval()
    
    for image_name in tqdm(os.listdir("datasets/coco/images/val2017")):
        compress_image(os.path.join("datasets/coco/images/val2017", image_name), os.path.join("datasets/coco_compressed/images/val2017", image_name), model=compress_model)

    results = detection_model.val(data="coco_compressed.yaml")
    print(model_name[0] if model_name is not None else None, model_name[1] if model_name is not None else None, results.box.map, results.box.map50, results.box.map75)

  0%|          | 0/5000 [00:00<?, ?it/s]

YOLOv12x summary (fused): 283 layers, 59,135,744 parameters, 0 gradients, 199.0 GFLOPs


val: Scanning /data/mavrinti/neural-image-compression/datasets/coco_compressed/labels/val2017... 4952 images, 48 backgrounds, 0 corrupt: 100%|██████████| 5000/5000 [00:34<00:00, 145.49it/s]


val: New cache created: /data/mavrinti/neural-image-compression/datasets/coco_compressed/labels/val2017.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 313/313 [03:00<00:00,  1.73it/s]


                   all       5000      36335       0.65      0.475      0.524      0.349
                person       2693      10777      0.786      0.586      0.672      0.446
               bicycle        149        314      0.713      0.285      0.371      0.218
                   car        535       1918      0.717      0.384      0.454      0.261
            motorcycle        159        367      0.753      0.561      0.634      0.403
              airplane         97        143      0.718      0.762      0.813      0.626
                   bus        189        283      0.744      0.675      0.748      0.606
                 train        157        190      0.806      0.774       0.85      0.686
                 truck        250        414      0.653      0.367      0.456      0.302
                  boat        121        424      0.523      0.325       0.35      0.172
         traffic light        191        634      0.686      0.227      0.297      0.144
          fire hydran

---